In [1]:
%%writefile src/MyToken.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";

contract MyToken is ERC20 {
    constructor() ERC20("MyToken", "JSL"){
        _mint(msg.sender, 1000000000000000000000000);
        //_mint(msg.sender, 1000000*decimals());
    }
}

Writing src/MyToken.sol


In [2]:
!solc-windows.exe --optimize --combined-json abi,bin src/MyToken.sol > src/MyToken.json

Error: Source "@openzeppelin/contracts/token/ERC20/ERC20.sol" not found: File outside of allowed directories.
 --> src/MyToken.sol:4:1:
  |
4 | import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
  | ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



@openzeppelin 없어서 오류가 난다

디렉토리에서 
```npm install @openzeppelin/contracts```
로 @openzeppelin 가져옴

@openzeppelin 경로 재지정

In [5]:
!solc-windows.exe @openzeppelin/="C:\\Users\\chohj\\Code\\201810811\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyToken.sol > src/MyToken.json

In [7]:
!type src\MyToken.json

{"contracts":{"C://Users//chohj//Code//201810811//node_modules//@openzeppelin/contracts/token/ERC20/ERC20.sol:ERC20":{"abi":[{"inputs":[{"internalType":"string","name":"name_","type":"string"},{"internalType":"string","name":"symbol_","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"a

In [8]:
%%writefile src/MyTokenTest.js
var _abiJson = require('./MyToken.json');
contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[4]); //reading src/MyToken.sol:MyToken

Writing src/MyTokenTest.js


In [9]:
!node src/MyTokenTest.js

- contract name:  src/MyToken.sol:MyToken


In [10]:
%%writefile src/MyTokenTest.js
var _abiJson = require('./MyToken.json');
contractName=Object.keys(_abiJson.contracts);
console.log("- contract name: ", contractName[4]);

_abi=_abiJson.contracts[contractName[4]].abi
_bin=_abiJson.contracts[contractName[4]].bin
console.log("- ABI: ", _abi);
console.log("- Bytecode: ", _bin);

Overwriting src/MyTokenTest.js


In [11]:
!node src/MyTokenTest.js

- contract name:  src/MyToken.sol:MyToken
- ABI:  [
  { inputs: [], stateMutability: 'nonpayable', type: 'constructor' },
  {
    anonymous: false,
    inputs: [ [Object], [Object], [Object] ],
    name: 'Approval',
    type: 'event'
  },
  {
    anonymous: false,
    inputs: [ [Object], [Object], [Object] ],
    name: 'Transfer',
    type: 'event'
  },
  {
    inputs: [ [Object], [Object] ],
    name: 'allowance',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  },
  {
    inputs: [ [Object], [Object] ],
    name: 'approve',
    outputs: [ [Object] ],
    stateMutability: 'nonpayable',
    type: 'function'
  },
  {
    inputs: [ [Object] ],
    name: 'balanceOf',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  },
  {
    inputs: [],
    name: 'decimals',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function'
  },
  {
    inputs: [ [Object], [Object] ],
    name: 'decreaseAllowance',
    outputs: [ [Obje

In [14]:
%%writefile src/MyTokenDeployAbiBinFromFile.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName[4]); //or console.log(contractName);

//_abi=_abiBinJson.contracts[contractName[4]].abi
//_bin=_abiBinJson.contracts[contractName[4]].bin


//contractName=Object.keys(_abiJson.contracts); // reading ['src//Timer.sol:Timer'];
//console.log("- contract name: ", contractName[4]); //or console.log(contractName);
_abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));    //JSON parsing needed!!
//_abiArray=JSON.stringify(_abi)
_bin="0x"+_abiBinJson.contracts[contractName[4]].bin;
//console.log("- ABI: " + JSON.stringify(_abiArray));
//console.log("- Bytecode: " + _bin);

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/MyTokenDeployAbiBinFromFile.js


In [15]:
!node src/MyTokenDeployAbiBinFromFile.js

- contract name:  src/MyToken.sol:MyToken
Deploying the contract from 0x15780bf31d2d65095C58ACc1995B06Ec1189dCDD
hash: 0xd6182972b598e76a0fc581e073071f3c6c0edfd2634afd5caad171027c9ca024
---> The contract deployed to: 0x4C4C59Fe78da1E40b8b32B5FE321680Ab22f16b2


In [18]:
%%writefile src/MyTokenUse.js
var Web3=require('web3');
var _abiBinJson = require('./MyToken.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts); 
console.log("- contract name: ", contractName[4]);// reading ['src//Timer.sol:Timer']
var _abiArray=JSON.parse(JSON.stringify(_abiBinJson.contracts[contractName[4]].abi));    //JSON parsing needed!!

var _test = new web3.eth.Contract(_abiArray, '0x4C4C59Fe78da1E40b8b32B5FE321680Ab22f16b2');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    const name = await _test.methods.name().call();
    console.log("name: " + name);
    const decimals = await _test.methods.decimals().call();
    console.log("decimals: " + decimals);
    const value = await _test.methods.totalSupply().call();
    console.log("totalSupply: " + value);

    console.log("-----1 before transfer-----");
    console.log("1-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("1-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    await _test.methods.transfer(accounts[1], 1)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("-----2 after transfer-----");
    console.log("2-1 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("2-2 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    console.log("-----3 after approve and transferFrom-----");    
    var addrTo = accounts[0];
    console.log("3-1 allowanceOfAddrTo: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.approve(addrTo, 111)
        .send({from: accounts[0], gas: 100000});
    console.log("3-2 allowanceOfAddrTo 111: "+ await _test.methods.allowance(accounts[0], addrTo).call());
    await _test.methods.transferFrom(addrTo, accounts[1], 1)
        .send({from: accounts[0], gas: 3641240, gasPrice: '1000000000'})
    console.log("3-3 balanceOfAccount0: " + await _test.methods.balanceOf(accounts[0]).call());
    console.log("3-4 balanceOfAccount1: " + await _test.methods.balanceOf(accounts[1]).call());
    //const value = await _test.methods.myFunction()
    //    .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    //console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/MyTokenUse.js


In [19]:
!node src/MyTokenUse.js

- contract name:  src/MyToken.sol:MyToken
Account: 0x15780bf31d2d65095C58ACc1995B06Ec1189dCDD
Balance before: 999998748350000000000
name: MyToken
decimals: 18
totalSupply: 1000000000000000000000000
-----1 before transfer-----
1-1 balanceOfAccount0: 1000000000000000000000000
1-2 balanceOfAccount1: 0
-----2 after transfer-----
2-1 balanceOfAccount0: 999999999999999999999999
2-2 balanceOfAccount1: 1
-----3 after approve and transferFrom-----
3-1 allowanceOfAddrTo: 0
3-2 allowanceOfAddrTo 111: 111
3-3 balanceOfAccount0: 999999999999999999999998
3-4 balanceOfAccount1: 2
Balance after: 999998562118000000000
Balance diff: 186231999954944


# NFT

In [20]:
%%writefile src/nftHelloUri.json
{
    "name": "GRBurger",
    "description": "Gordon Ramsay Burger is delicious",
    "image": "https://ipfs.io/ipfs/QmVenVdZQ5XmXYhHtDFy1o8hZzefp6tXMgptGrzT2mZDN6"
}

Writing src/nftHelloUri.json


In [21]:
%%writefile src/MyNFT.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/utils/Counters.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/utils/Counters.sol";

contract MyNFT is ERC721 {
	using Counters for Counters.Counter;
	Counters.Counter private _tokenIds;	//NFT id
	
	constructor() ERC721("Hello DApp Screens", "JSL") {}
	// override
	// does not save toeknURI in the state var
	function _baseURI() internal pure override returns (string memory) {
		return "https://ipfs.io/ipfs/QmbdwcmzYacLZHLBAhAsaJm2aPV1bzBjFjweD1UgZzkEtA";
	}

    function mint(address to) public returns(uint256) {
        //require(_tokenIds.current() < 3); //only 3 to mint
        _tokenIds.increment(); // add 1 by minting
        uint256 newTokenId = _tokenIds.current();
        _mint(to, newTokenId); // or _safeMint(to, newTokenId);

        return newTokenId;
    }
}

Writing src/MyNFT.sol


In [23]:
!solc-windows.exe @openzeppelin/="C:\\Users\\chohj\\Code\\201810811\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyNFT.sol > src/MyNFT.json

In [24]:
%%writefile src/MyNFT.sol
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@openzeppelin/contracts/token/ERC721/ERC721.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/token/ERC721/ERC721.sol";
import "@openzeppelin/contracts/utils/Counters.sol";
//import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/v4.4.0/contracts/utils/Counters.sol";

contract MyNFT is ERC721 {
	using Counters for Counters.Counter;
	Counters.Counter private _tokenIds;	//NFT id
	
	constructor() ERC721("GRBurger", "GRB") {}
	// override
	// does not save toeknURI in the state var
	function _baseURI() internal pure override returns (string memory) {
		return "https://ipfs.io/ipfs/QmVenVdZQ5XmXYhHtDFy1o8hZzefp6tXMgptGrzT2mZDN6";
	}

    function mint(address to) public returns(uint256) {
        //require(_tokenIds.current() < 3); //only 3 to mint
        _tokenIds.increment(); // add 1 by minting
        uint256 newTokenId = _tokenIds.current();
        _mint(to, newTokenId); // or _safeMint(to, newTokenId);

        return newTokenId;
    }
}

Overwriting src/MyNFT.sol


In [25]:
!solc-windows.exe @openzeppelin/="C:\\Users\\chohj\\Code\\201810811\\node_modules\\@openzeppelin\\" --optimize --combined-json abi,bin src/MyNFT.sol > src/MyNFT.json